In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from pandas_datareader import data
from pandas.tseries.offsets import *
%matplotlib inline

companies = ["AMZN", "NKE", "LULU", "MSFT", "EA", "GOOGL"]
start = "2007-12-31"
end = pd.datetime.now()
wts = [.166667,.166667, .166667,.166667,.166667,.166667]
index = "^GSPC"

core = data.DataReader(name = companies, data_source="yahoo", start = start, end = end)
benchmark = data.DataReader(name = index, data_source="yahoo", start = start, end = end)
core.tail(5)
benchmark.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-11-04,3085.199951,3074.870117,3078.959961,3078.270020,4146850000,3078.270020
2019-11-05,3083.949951,3072.149902,3080.800049,3074.620117,4486130000,3074.620117
2019-11-06,3078.340088,3065.889893,3075.100098,3076.780029,4458190000,3076.780029
2019-11-07,3097.770020,3080.229980,3087.020020,3085.179932,4144640000,3085.179932
2019-11-08,3093.090088,3073.580078,3081.250000,3093.080078,3499150000,3093.080078


In [5]:
#manually calcing one day return to validate the pct_change works
core["Adj Close"]["AMZN"]["2008-01-02"] / core["Adj Close"]["AMZN"]["2007-12-31"] - 1
core["Adj Close"]["EA"]["2008-01-02"] / core["Adj Close"]["EA"]["2007-12-31"] - 1
sum(wts)
benchmark["Adj Close"]["2019-11-08"]

3093.080078125

In [6]:
#Extracting Columns- First extract the Adjusted Close prices and then add brackets to extract just EA's adj close price
#add the pct_change method using 1 day to get series of daily returns
core["Adj Close"]
core["High"]["EA"]
core["Adj Close"]["EA"].pct_change(1)
core["Adj Close"]["EA"].pct_change(1).tail()

Date
2019-11-04    0.029906
2019-11-05    0.001121
2019-11-06    0.004173
2019-11-07   -0.014594
2019-11-08    0.010696
Name: EA, dtype: float64

In [10]:
#assign a variable to the daily returns for EA and for all other stocks
EA_returns = core["Adj Close"]["EA"].pct_change(1)
AMZN_returns = core["Adj Close"]["AMZN"].pct_change(1)
GOOGL_returns = core["Adj Close"]["GOOGL"].pct_change(1)
NKE_returns = core["Adj Close"]["NKE"].pct_change(1)
LULU_returns = core["Adj Close"]["LULU"].pct_change(1)
MSFT_returns = core["Adj Close"]["MSFT"].pct_change(1)
IVV_returns = benchmark["Adj Close"].pct_change(1)
EA_returns.head()
#series of daily returns for Amazon that excludes row 0 the Null value - 2 ways to accomplish this
core["Adj Close"]["AMZN"].pct_change(1)[1:]
core["Adj Close"]["AMZN"].pct_change(1).dropna()
NKE_returns.tail()
IVV_returns.dropna()
LULU_returns.tail()

Date
2019-11-04   -0.020978
2019-11-05   -0.010892
2019-11-06    0.032932
2019-11-07    0.010362
2019-11-08    0.006459
Name: LULU, dtype: float64

In [11]:
#create a dataframe that includes all daily returns for all stocks
core["Adj Close"].pct_change(1).dropna()
#convert the daily returns to a percent format (multiply by 100)
core["Adj Close"].pct_change(1).dropna() * 100
#assign a variable to the daily returns for each stock
daily_returns = core["Adj Close"].pct_change(1).dropna()
daily_returns.head(10)

Symbols,AMZN,EA,GOOGL,LULU,MSFT,NKE
Date,,,,,,
2008-01-02,0.038968,-0.028249,-0.009096,-0.053832,-0.010674,-0.014944
2008-01-03,-0.010805,-0.010923,0.000204,-0.001339,0.004259,-0.009008
2008-01-04,-0.067430,-0.029035,-0.041338,-0.094281,-0.027990,-0.015468
2008-01-07,0.000338,-0.008806,-0.011796,-0.051554,0.006690,0.005993
2008-01-08,-0.010583,-0.009809,-0.027062,-0.039532,-0.033516,-0.004508
2008-01-09,-0.030269,-0.004673,0.034068,-0.079339,0.029596,0.002264
2008-01-10,-0.011265,0.008826,-0.009905,0.021177,-0.003194,0.009198
2008-01-11,-0.037740,-0.016009,-0.013112,-0.047523,-0.012234,-0.029421
2008-01-14,0.022077,0.008702,0.024395,0.042637,0.014155,0.000494


In [12]:
#calculate weighted returns for to get the portfolio return
daily_returns * wts  #multiply the weights variable by your daily returns
weighted_returns = (daily_returns * wts)  #assign a variable 
weighted_returns
daily_returns.tail()
weighted_returns.tail()
weighted_returns.astype(float).tail()
weighted_returns.sum(axis = "columns").tail()   #sum up each weighted return to calculate the portfolio return
pd.DataFrame(weighted_returns.sum(axis = 1))   #turn that series of portfolio returns into a dataframe
portfolio_returns = pd.DataFrame(weighted_returns.sum(axis = 1))    #assign the variable to the portfolio returns dataframe
portfolio_returns.rename(columns = {0 : "Portfolio Returns"}, inplace = True)  #change the name of the column and set it inplace
portfolio_returns
portfolio_returns.head()

,Portfolio Returns
Date,
2008-01-02,-0.012971
2008-01-03,-0.004602
2008-01-04,-0.045924
2008-01-07,-0.009856
2008-01-08,-0.020835


In [13]:
benchmark_returns = pd.DataFrame(IVV_returns).dropna()
benchmark_returns.rename(columns = {"Adj Close" : "SP500 BM"}, inplace = True)
benchmark_returns.head()

,SP500 BM
Date,
2008-01-02,-0.014438
2008-01-03,0.000000
2008-01-04,-0.024552
2008-01-07,0.003223
2008-01-08,-0.018352


In [ ]:
#cumulative return of the portfolio 
#use the cumprod method and create a new variable for the cumulative returns
(portfolio_returns + 1)
(portfolio_returns + 1).cumprod() * 1000
growth1K = (portfolio_returns + 1).cumprod()
growthBM = (benchmark_returns + 1).cumprod()
#growth1K.plot()  #to plot just the portfolio growth chart
#growthBM.plot()  #to plot just the benchmark growth chart
growth1K.join(growthBM) #join the two dataframes
growth_portfolio_vs_bm = growth1K.join(growthBM)  #create a new variable for joined data set
growth_portfolio_vs_bm.plot()  #Plot the two together after merging data sets
#more efficient way to do the same below

In [ ]:
#calculate inception to date APR
portfolio_returns.count()  #total count of daily returns
portfolio_returns.iloc[1]
portfolio_returns.tail()
portfolio_returns.iloc[2972]
(portfolio_returns + 1).cumprod()
port_vs_bm_daily = portfolio_returns.join(benchmark_returns) #this line and the one right below - get u same result as code in cell above
(port_vs_bm_daily + 1).cumprod().tail()  #the last two lines are the same as the code in cell above 

cumulative_returns = (port_vs_bm_daily + 1).cumprod()

growth_portfolio_vs_bm.tail()  #same result as line above
cumulative_returns.tail()

In [ ]:
#annualized returns since inception
portfolio_returns.count()
365 / 2972
365 / portfolio_returns.count()
cumulative_returns.iloc[2972]  ** (365 / 2973) - 1  #ITD APR
SI_APR = cumulative_returns.iloc[2972]  ** (365 / 2973) - 1
SI_APR

In [ ]:
port_vs_bm_daily.std()
port_vs_bm_daily.head().std() #Standard deviation of returns 
np.sqrt(12)  #numpy square root of 12
port_vs_bm_daily.std() * np.sqrt(12) #annualized volatility 
annualized_vol = port_vs_bm_daily.std() * np.sqrt(12)
annualized_vol

##### Now calculate monthly returns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from pandas_datareader import data
from pandas.tseries.offsets import *
%matplotlib inline

companies = ["AMZN", "NKE", "LULU", "MSFT", "EA", "GOOGL"]
start = "2007-12-31"
end = pd.datetime.now()
wts = [.166667,.166667, .166667,.166667,.166667,.166667]
index = "^GSPC"

core = data.DataReader(name = companies, data_source="yahoo", start = start, end = end)
benchmark = data.DataReader(name = index, data_source="yahoo", start = start, end = end)
core.tail(5)
benchmark.tail()

In [ ]:
#calculate monthly returns
core["Adj Close"].pct_change(1)

In [ ]:
core["Adj Close"].head()

In [ ]:
amazon_cumulative = (AMZN_returns + 1).cumprod()
amazon_cumulative.iloc[2973] ** (365 / 2973) - 1
amazon_cumulative

In [ ]:
#Figuring out how to calculate monthly returns
daily_returns.asfreq('m')
daily_returns.index + pd.tseries.offsets.MonthEnd()
daily_returns.loc['2018-01-31']
daily_close = core["Adj Close"]  #daily_close variable for all the daily adjusted closes for each stock
daily_close.truncate(before = "2008-08-20")  #can use truncate to strip down dataframe
daily_close.asfreq('BM')  #set the frequency of the daily close table to last business day of the month
monthly_close = daily_close.asfreq('BM')
monthly_close
monthly_close.pct_change(1)  #periods will be one in this case because it looks back 1 row which is previous month end
monthly_returns = monthly_close.pct_change(1)  #set the monthly returns to a new variable
cumulative_stock_returns = (monthly_returns + 1).cumprod()
cumulative_stock_returns.plot()